In [1]:
import pandas as pd
import sqlalchemy
from routes import routes
from utils import graph_utils, station_manager
from tqdm import tqdm

In [2]:
# import stations
stations = routes.extracts('../data/processed/stations.gpkg')
remover = routes.select_criteria(stations) # filter stations (same criteria as routes)
stations = station_manager.remove_stations(stations, remover) # remover as estações deletadas
# make graph
G = graph_utils.generate_graph(stations)


In [3]:
engine = sqlalchemy.create_engine('sqlite:///../data/processed/database.db')

def import_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

query = """SELECT 
    o.data,
    o.id,
    o.TU,
    o.ValorTKU,
    f.ferrovia,
    f.cliente,
    f.mercadoria,
    f.origem,
    f.destino,
    f.dist_media,
    f.intermed,
    f.len_Dijkstra,
    f.rota
FROM 
    ocorrencias o
LEFT JOIN 
    fluxos_rotas_2017 f
ON 
    o.id = f.id
WHERE 
    o.data >= '2024-01-01' and o.data <= '2024-12-01';
"""
fluxos = pd.read_sql(query, engine)

In [4]:
def nodes_to_edges(df):

    # Verificar monotonicidade em cada grupo
    for name, group in df.groupby('linha'):
        if not group['sequencia'].is_monotonic_increasing:
            print(f'sequencia not increasing in linha {name}')
            return None
    
    edges = df.copy()
    edges['estacao_anterior'] = edges['estacao'].shift()
    edges['geometry_anterior'] = edges['points_geometry'].shift() # essa tem que ser a points_geometry
    edges = edges[edges['sequencia'] != 1].reset_index(drop=True)
    edges['sequencia'] -= 1


    return edges

edges = nodes_to_edges(stations)

edges = edges[['ferrovia','linha','estacao_anterior','estacao', 'sequencia', 'extensao', 'geometry_anterior','points_geometry']].copy()
edges

,ferrovia,linha,estacao_anterior,estacao,sequencia,extensao,geometry_anterior,points_geometry
0,RMP,Alça Alumínio-Capricórnio,ZAL,ZPS,1,4.700,POINT (-47.258995 -23.536671),POINT (-47.228199 -23.533678)
1,RMP,Alça Boa Vista Nova - Boa Vista Velha,ZBL,ZBV,1,1.300,POINT (-47.139936 -22.877123),POINT (-47.142709 -22.886679)
2,RMP,Alça Canguera,ZMK,ZKE,1,6.700,POINT (-47.183361 -23.544556),POINT (-47.17005 -23.572923)
3,RMP,Alça Pantojo-Capricórnio,ZXY,ZPS,1,0.750,POINT (-47.23033 -23.541505),POINT (-47.228199 -23.533678)
4,RMP,Alça de Campinas,ZBL,Z51,1,1.960,POINT (-47.139936 -22.877123),POINT (-47.134899 -22.889949)
...,...,...,...,...,...,...,...,...
1904,FCA,Uberaba - Araguari,ZZB,ZYE,5,10.929,POINT (-48.109098 -19.287937),POINT (-48.12125 -19.193632)
1905,FCA,Uberaba - Araguari,ZYE,ZHU,6,21.900,POINT (-48.12125 -19.193632),POINT (-48.158853 -19.022341)
1906,FCA,Uberaba - Araguari,ZHU,ZUL,7,21.370,POINT (-48.158853 -19.022341),POINT (-48.246125 -18.887973)
1907,FCA,Uberaba - Araguari,ZUL,ZHF,8,24.808,POINT (-48.246125 -18.887973),POINT (-48.25132 -18.749177)


In [6]:
for index, row in tqdm(edges.iterrows()):
    _, path = graph_utils.must_pass(G, [row['estacao_anterior'], row['estacao']])
    
    x = 0
    for i, r in fluxos.iterrows():

        x += graph_utils.find_tu(G, r['rota'], path, r['TU'], row['ferrovia'])
    
    edges.at[index, 'TU'] = x


1909it [2:04:58,  3.93s/it] 


In [ ]:
#x = fluxos.apply(lambda row: graph_utils.find_tu(G, row['Rota'], path, row['TU'],'RMS'), axis=1)

In [7]:
edges

,ferrovia,linha,estacao_anterior,estacao,sequencia,extensao,geometry_anterior,points_geometry,TU
0,RMP,Alça Alumínio-Capricórnio,ZAL,ZPS,1,4.700,POINT (-47.258995 -23.536671),POINT (-47.228199 -23.533678),1800179.0
1,RMP,Alça Boa Vista Nova - Boa Vista Velha,ZBL,ZBV,1,1.300,POINT (-47.139936 -22.877123),POINT (-47.142709 -22.886679),3944356.0
2,RMP,Alça Canguera,ZMK,ZKE,1,6.700,POINT (-47.183361 -23.544556),POINT (-47.17005 -23.572923),0.0
3,RMP,Alça Pantojo-Capricórnio,ZXY,ZPS,1,0.750,POINT (-47.23033 -23.541505),POINT (-47.228199 -23.533678),0.0
4,RMP,Alça de Campinas,ZBL,Z51,1,1.960,POINT (-47.139936 -22.877123),POINT (-47.134899 -22.889949),608341.0
...,...,...,...,...,...,...,...,...,...
1904,FCA,Uberaba - Araguari,ZZB,ZYE,5,10.929,POINT (-48.109098 -19.287937),POINT (-48.12125 -19.193632),2597721.0
1905,FCA,Uberaba - Araguari,ZYE,ZHU,6,21.900,POINT (-48.12125 -19.193632),POINT (-48.158853 -19.022341),2597721.0
1906,FCA,Uberaba - Araguari,ZHU,ZUL,7,21.370,POINT (-48.158853 -19.022341),POINT (-48.246125 -18.887973),2597721.0
1907,FCA,Uberaba - Araguari,ZUL,ZHF,8,24.808,POINT (-48.246125 -18.887973),POINT (-48.25132 -18.749177),2597721.0
